In [7]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
%load_ext autoreload
%autoreload 2
DATA_DIR='data_nuclis'
DATA_DIR_DEV='data_nuclis_dev'
#test_file='2tg_E98p_211a001{0}.png'
from image_utils import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
generate_images_folder(DATA_DIR)
generate_images_folder(DATA_DIR_DEV)

folder data_nuclis/2tg_E103p_223a001
folder data_nuclis_dev/2tg_E98p_224a001


In [21]:
generate_images_folder_v2(DATA_DIR)
#generate_images_folder_v2(DATA_DIR_DEV)

df data_nuclis/2tg_E103p_223a001/2tg_E103p_223a001{0}.png
folder data_nuclis/2tg_E103p_223a001 0 0
folder data_nuclis/2tg_E103p_223a001 0 1
folder data_nuclis/2tg_E103p_223a001 1 0
folder data_nuclis/2tg_E103p_223a001 1 1
df data_nuclis/2tg_E98p_224a001/2tg_E98p_224a001{0}.png
folder data_nuclis/2tg_E98p_224a001 0 0
folder data_nuclis/2tg_E98p_224a001 0 1
folder data_nuclis/2tg_E98p_224a001 1 0
folder data_nuclis/2tg_E98p_224a001 1 1


In [2]:
X_train,Y_train,file_names_train=get_images_data(DATA_DIR)
X_dev,Y_dev,file_names_dev=get_images_data(DATA_DIR_DEV)

In [14]:
import tensorflow as tf
DEV_FILE='data_nuclis_dev/2tg_E98p_224a001/2tg_E98p_224a001.png'
TRAIN_FILE='data_nuclis/2tg_E103p_223a001/2tg_E103p_223a001.png'

nx=X_train.shape[0]
num_classes=1
X = tf.placeholder(tf.float32, [nx,None],name="X")
W = tf.Variable(tf.zeros([num_classes, nx]),name="W")
b = tf.Variable(tf.zeros([num_classes,1]),name="b")
Z = tf.matmul(W, X) + b
Ypred=tf.sigmoid(Z)
Y = tf.placeholder(tf.float32, [num_classes, None])
cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tf.transpose(Z),  labels = tf.transpose(Y)))#+tf.nn.l2_loss(W)
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cost)
correct_prediction = tf.equal(Y, tf.cast(tf.greater(Ypred,0.5),"float"))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
patches_dev,slices_dev=patches_of_image(DEV_FILE,40,40)
patches_train,slices_train=patches_of_image(TRAIN_FILE,40,40)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(100):
        _,j,a=sess.run([train_step,cost,accuracy],feed_dict={X:X_train,Y:Y_train})
        print i,"train",j,a
        print i,"test",sess.run([cost,accuracy],feed_dict={X:X_dev,Y:Y_dev})
    Ydev=sess.run(Ypred,feed_dict={X:patches_dev})
    Ytrain=sess.run(Ypred,feed_dict={X:patches_train})
    draw_rectangles(DEV_FILE,filter_rectangles(slices_dev[Ydev>=0.5],10))
    draw_rectangles(TRAIN_FILE,filter_rectangles(slices_train[Ytrain>=0.5],10))

0 train 0.693147 0.562077
0 test [0.64002037, 0.6982758]
1 train 0.680908 0.562077
1 test [0.6368705, 0.6982758]
2 train 0.67973 0.562077
2 test [0.63569611, 0.6982758]
3 train 0.678586 0.562077
3 test [0.63465297, 0.6982758]
4 train 0.677446 0.562077
4 test [0.63362181, 0.6982758]
5 train 0.676309 0.562077
5 test [0.63259465, 0.6982758]
6 train 0.675177 0.562077
6 test [0.6315707, 0.6982758]
7 train 0.674047 0.562077
7 test [0.63055021, 0.6982758]
8 train 0.672921 0.562077
8 test [0.62953293, 0.6982758]
9 train 0.671799 0.562077
9 test [0.62851894, 0.6982758]
10 train 0.67068 0.562077
10 test [0.62750828, 0.6982758]
11 train 0.669565 0.562077
11 test [0.6265009, 0.6982758]
12 train 0.668453 0.562077
12 test [0.62549675, 0.6982758]
13 train 0.667345 0.562077
13 test [0.62449586, 0.6982758]
14 train 0.66624 0.562077
14 test [0.62349826, 0.6982758]
15 train 0.665138 0.562077
15 test [0.62250388, 0.6982758]
16 train 0.66404 0.562077
16 test [0.62151271, 0.6982758]
17 train 0.662946 0.5620

In [3]:
import tensorflow as tf
DEV_FILE='data_nuclis_dev/2tg_E98p_224a001/2tg_E98p_224a001.png'
TRAIN_FILE='data_nuclis/2tg_E103p_223a001/2tg_E103p_223a001.png'
NUM_EPOCHS=3000

nx=X_train.shape[0]
num_classes=1
num_hidden1=4000
num_hidden2=500
with tf.variable_scope('test02') as scope:
    X = tf.placeholder(tf.float32, [nx,None],name="X")
    keep_prob = tf.placeholder(tf.float32)
    W1 = tf.get_variable("W1", [num_hidden1,nx], initializer = tf.contrib.layers.xavier_initializer(seed = 1),dtype=tf.float32)
    b1 = tf.get_variable("b1",dtype=tf.float32,initializer=tf.constant(np.zeros((num_hidden1,1)),dtype=tf.float32))
    W2 = tf.get_variable("W2", [num_hidden2,num_hidden1], initializer = tf.contrib.layers.xavier_initializer(seed = 1),dtype=tf.float32)
    b2 = tf.get_variable("b2",dtype=tf.float32,initializer=tf.constant(np.zeros((num_hidden2,1)),dtype=tf.float32))
    W3 = tf.get_variable("W3", [num_classes,num_hidden2], initializer = tf.contrib.layers.xavier_initializer(seed = 1),dtype=tf.float32)
    b3 = tf.get_variable("b3",dtype=tf.float32,initializer=tf.constant(np.zeros((num_classes,1)),dtype=tf.float32))
    Z1 = tf.matmul(W1, X) + b1
    A1=tf.nn.dropout(tf.nn.relu(Z1),keep_prob)
    Z2 = tf.matmul(W2, A1) + b2
    A2=tf.nn.dropout(tf.nn.relu(Z2),keep_prob)
    Z3 = tf.matmul(W3,A2)+b3
    Ypred=tf.sigmoid(Z3)
    Y = tf.placeholder(tf.float32, [num_classes, None])
    regul=tf.nn.l2_loss(W1)+tf.nn.l2_loss(W2)+tf.nn.l2_loss(W3)
    cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tf.transpose(Z3),  labels = tf.transpose(Y)))+0.003*regul
    #train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cost)
    train_step = tf.train.AdamOptimizer(0.00001).minimize(cost)
    correct_prediction = tf.equal(Y, tf.cast(tf.greater(Ypred,0.5),"float"))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
patches_dev,slices_dev=patches_of_image(DEV_FILE,40,40)
patches_train,slices_train=patches_of_image(TRAIN_FILE,40,40)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(NUM_EPOCHS):
        _,j,a=sess.run([train_step,cost,accuracy],feed_dict={X:X_train,Y:Y_train,keep_prob:0.5})
        if i%100==0:
            print i,"train",j,a
            print i,"test",sess.run([cost,accuracy],feed_dict={X:X_dev,Y:Y_dev,keep_prob:1.})
            Ydev=sess.run(Ypred,feed_dict={X:patches_dev,keep_prob:1.})
            Ytrain=sess.run(Ypred,feed_dict={X:patches_train,keep_prob:1.})
            draw_rectangles(DEV_FILE,filter_rectangles(slices_dev[Ydev>=0.5],30))
            draw_rectangles(TRAIN_FILE,filter_rectangles(slices_train[Ytrain>=0.5],20))
        #if i==2000:
        #    print "Canviant train_step"
        #    train_step=tf.train.GradientDescentOptimizer(0.0001).minimize(cost)

0 train 8.86087 0.489089
0 test [8.5396442, 0.6982758]
100 train 7.73098 0.910271
100 test [7.7137432, 0.90660912]
200 train 7.23465 0.942814
200 test [7.3099766, 0.90708804]
300 train 6.82512 0.946012
300 test [6.9223347, 0.90996158]
400 train 6.44129 0.948269
400 test [6.556386, 0.90660906]
500 train 6.07961 0.951467
500 test [6.2025352, 0.91187727]
600 train 5.73843 0.956922
600 test [5.8697162, 0.91618764]
700 train 5.41629 0.959932
700 test [5.5567245, 0.92073745]
800 train 5.11234 0.965199
800 test [5.262063, 0.9243294]
900 train 4.82514 0.970843
900 test [4.9917488, 0.92768186]
1000 train 4.557 0.975545
1000 test [4.7285337, 0.93199223]
1100 train 4.30599 0.975922
1100 test [4.4800267, 0.9363026]
1200 train 4.07018 0.978179
1200 test [4.2670331, 0.93103439]
1300 train 3.84625 0.983634
1300 test [4.042295, 0.93582368]
1400 train 3.63626 0.983634
1400 test [3.844888, 0.9370209]
1500 train 3.4418 0.985892
1500 test [3.6526983, 0.93941557]
1600 train 3.25677 0.987396
1600 test [3.46